In [1]:
import re
import pandas as pd
import numpy as np
from mechanize import Browser
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [2]:
#you need to manually run your search and add your URL resulting from the search, 
#checking name, seq, Pubmed ID and hemolytic activity.
text = """ web input"""
browser = Browser()
response = browser.open("https://webs.iiitd.edu.in/raghava/hemolytik/submitkey_target_search.php?ran=6203")
content = response.read()
#seperating the data into the correct labels
split = str(content, 'utf-8').split(sep='</td><td')
clean_split = split[7:]
seq =[]
toxic = []
PMID = []
for i in range(int((len(clean_split)/4))):
    seq_pos = ((2) + 4*i)
    toxic_pos = ((3)+ 4*i)
    PMID_pos = ((0) + 4*i)
    seq.append((clean_split[seq_pos]).upper())
    toxic.append(clean_split[toxic_pos])
    PMID.append(clean_split[PMID_pos])
#cleaning the labels
for n in range(len(seq)):
    seq[n] = seq[n].split(sep=">")[1]
    PMID[n] = PMID[n].split(sep = "\"")[1]
    temp_toxic = toxic[n].split(sep = "center>")[1]
    toxic[n] = temp_toxic.split(sep = "</td>")[0]
df_hemolytik = pd.DataFrame([seq ,PMID ,toxic], index = ['seq' ,'PMID' ,'toxic'])
df_hemolytik = df_hemolytik.transpose()

In [3]:
#automatic reading of the activity from string to floats
df_hemolytik['toxic2'] = np.nan
df_hemolytik['toxic3'] = "other"
df_hemolytik['toxic4'] = "other"
df_hemolytik['toxic5'] = "other"
for n in range(len(df_hemolytik)):
    if "-" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('-[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "±" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic']= re.sub('±[^>]+[%μµnm ]', '', df_hemolytik.toxic[n])
    if "MHC" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('MHC', '15% hem')
    if "No hemolysis" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('No hemolysis', '0% hem')
    if "Lethal" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('Lethal', '100% hem')
    if "not detected" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic'] = df_hemolytik.toxic[n].replace('not detected', '0% hem')
    if "M" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "M"
    if "μM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μM"
    if "µg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "μg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"
    if "µg/mL" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "μg/ml"        
    if "nM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "nM"
    if "mg/L" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/L"
    if "mg/ml" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mg/ml"        
    if "mM" in df_hemolytik.toxic[n]:
        df_hemolytik.loc[n, 'toxic3'] = "mM"
    df_hemolytik.toxic2[n] = re.findall(r"[-+]?\d*\.\d+|\d+", str(df_hemolytik.toxic[n]))
    if (len(df_hemolytik.toxic2[n])==2):
        df_hemolytik.loc[n, 'toxic4'] = (df_hemolytik.toxic2[n])[0]
        df_hemolytik.loc[n, 'toxic5'] = (df_hemolytik.toxic2[n])[1]
df_hemolytik

<ipython-input-3-361ccd1cd9f5>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hemolytik.toxic2[n] = re.findall(r"[-+]?\d*\.\d+|\d+", str(df_hemolytik.toxic[n]))
C:\Users\JJvRijn\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,seq,PMID,toxic,toxic2,toxic3,toxic4,toxic5
0,ALWMTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.4M,"[50, 1.4]",M,50,1.4
1,ALWMTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
2,ALWDTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =2.3M,"[50, 2.3]",M,50,2.3
3,ALWDTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =5M,"[50, 5]",M,50,5
4,ALWMTLLKKVLKAAAKAALKAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
...,...,...,...,...,...,...,...
2578,ZCRRLSYKQRSVTYCRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2579,ZDRRLSYKQRSVTYORGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2580,ZSRRLDYKQROVTYSRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2581,ZCRRLEYKQRKVTYCRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100


In [4]:
#removing peptides that could not have their activity added correctly
unvalid1= []
unvalid2= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic3[n]== 'other')):
        unvalid1.append(n)
    if ((df_hemolytik.toxic4[n]== 'other')):
        unvalid2.append(n)
unvalid = unvalid1 + list(set(unvalid2) - set(unvalid1))
df_hemolytik = df_hemolytik.drop(unvalid)
df_hemolytik.reset_index(drop=True)

,seq,PMID,toxic,toxic2,toxic3,toxic4,toxic5
0,ALWMTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.4M,"[50, 1.4]",M,50,1.4
1,ALWMTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
2,ALWDTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =2.3M,"[50, 2.3]",M,50,2.3
3,ALWDTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =5M,"[50, 5]",M,50,5
4,ALWMTLLKKVLKAAAKAALKAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
...,...,...,...,...,...,...,...
2136,ZCRRLSYKQRSVTYCRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2137,ZDRRLSYKQRSVTYORGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2138,ZSRRLDYKQROVTYSRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100
2139,ZCRRLEYKQRKVTYCRGR,http://www.ncbi.nlm.nih.gov/pubmed/16235231,<10% hemolysis at 100µM,"[10, 100]",μM,10,100


In [5]:
#remove sequances that contain non amino acid characters as the following step wont work otherwise
df_hemolytik = df_hemolytik.reset_index(drop=True)
NAA = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "U",
       "T", "W", "Y", "V", " "]
unvalid =[]
for j in df_hemolytik.index:
    if (not all ([NAA_seq in NAA for NAA_seq in df_hemolytik.seq[j]])):
        unvalid.append(j)
df_hemolytik = df_hemolytik.drop(unvalid)
df_hemolytik.reset_index(drop=True)

,seq,PMID,toxic,toxic2,toxic3,toxic4,toxic5
0,ALWMTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.4M,"[50, 1.4]",M,50,1.4
1,ALWMTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
2,ALWDTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =2.3M,"[50, 2.3]",M,50,2.3
3,ALWDTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =5M,"[50, 5]",M,50,5
4,ALWMTLLKKVLKAAAKAALKAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2
...,...,...,...,...,...,...,...
1965,IWRIFR,http://www.ncbi.nlm.nih.gov/pubmed/22699557,15% hem >256μM,"[15, 256]",μM,15,256
1966,IWRIFRRIF,http://www.ncbi.nlm.nih.gov/pubmed/22699557,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4
1967,FGVLAKVAAHVVPAIAEHF,http://www.ncbi.nlm.nih.gov/pubmed/15298176,0% hemolysis upto 100µM,"[0, 100]",μM,0,100
1968,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,http://www.ncbi.nlm.nih.gov/pubmed/16460023,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6


In [6]:
df_hemolytik = df_hemolytik.reset_index(drop=True)
#types of units that need conversion "μg/ml", "nM", "M", "mM", "mg/L", "mg/ml"
glob = GlobalDescriptor(df_hemolytik.seq.tolist())
glob.calculate_MW(amide=True)
df_hemolytik['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask0 = df_hemolytik["toxic3"] == 'µg/ml'
mask1 = df_hemolytik["toxic3"] == 'mg/L'
mask2 = df_hemolytik["toxic3"] == 'mg/ml'
mask3 = df_hemolytik["toxic3"] == 'μM'
mask4 = df_hemolytik["toxic3"] == 'nM'
mask5 = df_hemolytik["toxic3"] == 'mM'
mask6 = df_hemolytik["toxic3"] == 'M'
#calculate the µM
df_hemolytik["CONCENTRATION_µM"] = 0
df_hemolytik.loc[mask0, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask0])*1000) / df_hemolytik[mask0]['MW']
df_hemolytik.loc[mask1, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask1]).astype(float)*1000) / df_hemolytik[mask1]['MW']
df_hemolytik.loc[mask2, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask2]).astype(float)*1000000) / df_hemolytik[mask2]['MW']
df_hemolytik.loc[mask4, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask4]).astype(float)/1000)
df_hemolytik.loc[mask5, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask5]).astype(float)*1000)
df_hemolytik.loc[mask6, "CONCENTRATION_µM"] = (
    (df_hemolytik["toxic5"][mask6]).astype(float)*1000000)
#fill in values already in µM
df_hemolytik.loc[mask3, "CONCENTRATION_µM"] = df_hemolytik["toxic5"][mask3].astype(float)
df_hemolytik

,seq,PMID,toxic,toxic2,toxic3,toxic4,toxic5,MW,CONCENTRATION_µM
0,ALWMTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.4M,"[50, 1.4]",M,50,1.4,2849.54,1.4e+06
1,ALWMTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2,2850.52,1.2e+06
2,ALWDTLLKKVLKAAAKAALNAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =2.3M,"[50, 2.3]",M,50,2.3,2833.43,2.3e+06
3,ALWDTLLKKVLKAAAKAALDAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =5M,"[50, 5]",M,50,5,2834.41,5e+06
4,ALWMTLLKKVLKAAAKAALKAVLVGANA,http://www.ncbi.nlm.nih.gov/pubmed/10660589,LC<sub>50</sub> =1.2M,"[50, 1.2]",M,50,1.2,2863.61,1.2e+06
...,...,...,...,...,...,...,...,...,...
1965,IWRIFR,http://www.ncbi.nlm.nih.gov/pubmed/22699557,15% hem >256μM,"[15, 256]",μM,15,256,889.10,256
1966,IWRIFRRIF,http://www.ncbi.nlm.nih.gov/pubmed/22699557,15% hem =101.4μM,"[15, 101.4]",μM,15,101.4,1305.62,101.4
1967,FGVLAKVAAHVVPAIAEHF,http://www.ncbi.nlm.nih.gov/pubmed/15298176,0% hemolysis upto 100µM,"[0, 100]",μM,0,100,1975.35,100
1968,FKIKASKVLDKFGKIVGKVLKQLKKVSAVAKV,http://www.ncbi.nlm.nih.gov/pubmed/16460023,LC<sub>50</sub>=12.6µM,"[50, 12.6]",μM,50,12.6,3498.40,12.6


In [7]:
#HAPPENN formating of positive/negative toxicity can be changed to other parameters
#happenn formating
df_hemolytik["toxic6"] = "~"
for v in range(len(df_hemolytik)):
    if float(df_hemolytik["toxic4"][v]) <= 5:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 30:
            df_hemolytik.loc[v, "toxic6"] = 0        
    if 5 <= float(df_hemolytik["toxic4"][v]) <= 10:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 60:
            df_hemolytik.loc[v, "toxic6"] = 0   
    if 10 <= float(df_hemolytik["toxic4"][v]) <= 15:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 90:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 15 <= float(df_hemolytik["toxic4"][v]) <= 20:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 120:
            df_hemolytik.loc[v, "toxic6"] = 0
    if 20 <= float(df_hemolytik["toxic4"][v]) <= 25:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 25 <= float(df_hemolytik["toxic4"][v]) <= 30:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 180:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 30 <= float(df_hemolytik["toxic4"][v]) <= 35:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 210:
            df_hemolytik.loc[v, "toxic6"] = 0         
    if 35 <= float(df_hemolytik["toxic4"][v]) <= 40:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 240:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if 40 <= float(df_hemolytik["toxic4"][v]) <= 45:
        if (df_hemolytik["CONCENTRATION_µM"][v]) >= 270:
            df_hemolytik.loc[v, "toxic6"] = 0  
    if float(df_hemolytik["toxic4"][v]) >= 50:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 300:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 55:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 330:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 60:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 360:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 65:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 390:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 70:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 420:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 75:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 450:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 80:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 480:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 85:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 510:
            df_hemolytik.loc[v, "toxic6"] = 1 
    if float(df_hemolytik["toxic4"][v]) >= 90:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 540:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 95:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 570:
            df_hemolytik.loc[v, "toxic6"] = 1
    if float(df_hemolytik["toxic4"][v]) >= 100:
        if (df_hemolytik["CONCENTRATION_µM"][v]) <= 600:
            df_hemolytik.loc[v, "toxic6"] = 1 

In [8]:
#removing peptides that are in the grey space between toxic and non-toxic
unvalid1= []
for n in range(len(df_hemolytik)):
    if ((df_hemolytik.toxic6[n]== '~')):
        unvalid1.append(n)
df_hemolytik = df_hemolytik.drop(unvalid1)
df_hemolytik = df_hemolytik.reset_index(drop=True)

In [9]:
#This one combines duplicates of peptides and see if they match, when they do not they get removed
peptides = df_hemolytik["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(df_hemolytik)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if df_hemolytik["seq"][a] == peptides[n]:
            toxic.append(df_hemolytik["toxic6"][a])
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1
unvalid1= []
for n in range(len(final_data)):
    if ((final_data.toxic[n]== '~')):
        unvalid1.append(n)
final_data = final_data.drop(unvalid1)
final_data = final_data.reset_index(drop=True)

In [10]:
final_data.to_csv("###your perfered data location###",index = False, header=True)

In [11]:
#assesing how well the data can be seperated and what ML models perform well on the data
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from lazypredict.Supervised import LazyClassifier
from DNN_features import *
test2 = AAC_df(final_data.seq)
train_X, val_X, train_y, val_y = train_test_split(test2, final_data.toxic, random_state = 445)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
#models_train,predictions_train = clf.fit(train_X, train_X, train_y, train_y)
models,predictions = clf.fit(train_X, val_X, train_y, val_y)
models

C:\Users\JJvRijn\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


ModuleNotFoundError: No module named 'DNN_features'

In [ ]:
final_data.toxic.value_counts()